In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model
from hmpai.pytorch.generators import SAT1Dataset
from hmpai.data import SAT1_STAGES_ACCURACY, COMMON_STAGES, AR_STAGES
from hmpai.visualization import plot_confusion_matrix
from hmpai.normalization import *
from torchinfo import summary
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
from mne.io import read_info
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [3]:
set_global_seed(42)
data_path_sat1 = Path("../data/ar/stage_data.nc")
dataset_sat1 = xr.load_dataset(data_path_sat1)

In [4]:
# Split before merging to ensure equal participant distribution across datasets and normalization within dataset
shape_topological = False
train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
    dataset_sat1, 60, norm_min1_to_1
)

In [5]:
train_dataset = SAT1Dataset(
    train_data_sat1, shape_topological=shape_topological, labels=AR_STAGES
)
val_dataset = SAT1Dataset(
    val_data_sat1, shape_topological=shape_topological, labels=AR_STAGES
)
test_dataset = SAT1Dataset(
    test_data_sat1, shape_topological=shape_topological, labels=AR_STAGES
)

In [9]:
model = TransformerModel(
    len(train_data_sat1.channels), 8, 512, 6, len(AR_STAGES)
)

train_and_test(
    model,
    train_dataset,
    test_dataset,
    val_dataset,
    logs_path=Path("../logs/"),
    workers=0,
    batch_size=128,
)

/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/69 [00:00<?, ? batch/s]

ValueError: Expected input batch_size (46080) to match target batch_size (38400).